In [ ]:
import cv2
import sys
import os
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def im_show(image):
    plt.imshow(image)
    plt.show()


face_txt = 'datas/recog/faceid_player_playerNum_LNBGvsZJCZ_615.ts.txt'
#track_file = '/home/avs/Codes/PaddleDetection/output/LNBGvsZJCZ_615.txt'
track_file = '/home/avs/Codes/ByteTrack/YOLOX_outputs/yolox_x_mix_det/track_vis/2023_07_04_10_38_42.txt'

def read_face_file(face_file):
    frame_2_face_list = {}
    with open(face_txt, 'r') as fin:
        for idx, line in enumerate(fin.readlines()):
            frame_idx, x1, y1, x2, y2, face_id, score = line.split(",")
            face_list = frame_2_face_list.get(frame_idx, [])
            face_box = list(map(float, [face_id, x1, y1, x2, y2]))
            face_list.append(face_box)
            frame_2_face_list[frame_idx] = face_list
    return frame_2_face_list

def read_track_file(track_file):
    frame_2_person_list = {}
    person_2_frame2 = {}
    with open(track_file, 'r') as fin:
        for line in fin.readlines():
            frame_idx, person_id, x, y, w, h, score, _, _, _ = line.split(",")
            person_list = frame_2_person_list.get(frame_idx, [])
            person_box = list(map(float, [person_id, x, y, w, h, score]))
            person_list.append(person_box)
            frame_2_person_list[frame_idx] = person_list
    return frame_2_person_list
       
    
def show_trace():
    frame_2_face_list = read_face_file(face_txt)
    frame_2_person_list = read_track_file(track_file)

    idx = 0
    for frame_idx in frame_2_face_list:
        idx += 1
        if idx >= 10:
            break
        frame = cv2.imread('./datas/LNBGvsZJCZ_615.ts_imgs/{}.jpg'.format(frame_idx))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        face_list = frame_2_face_list[frame_idx]
        face_id, x1, y1, x2, y2 = face_list[0]

        if frame_idx not in frame_2_person_list:
            print("== ignore ", frame_idx)
            continue
        person_list = frame_2_person_list[frame_idx]


        cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), (0,255,0) , 2)
        cv2.putText(frame, str(face_id), (int(x1-10),int(y1-10)), font, 1, (255,0,0), 3)


        font = cv2.FONT_HERSHEY_SIMPLEX
        for person in person_list:
            person_id = person[0]
            person_x1, person_y1, person_w, person_h = person[1:5]
            cv2.rectangle(frame, (int(person_x1),int(person_y1)), (int(person_x1+person_w),int(person_y1+person_h)), (0,0,255) , 2)
            cv2.putText(frame, str(person_id), (int(person_x1-10),int(person_y1-10)), font, 1, (255,0,0), 3)

        im_show(frame)
        

In [ ]:
def get_color(idx):
    idx = idx * 3
    color = ((37 * idx) % 255, (17 * idx) % 255, (29 * idx) % 255)
    return color


video_file = '/home/avs/Downloads/LNBGvsZJCZ_615.ts'
capture = cv2.VideoCapture(video_file)
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(capture.get(cv2.CAP_PROP_FPS))

output_dir = 'output'
out_path = os.path.join(output_dir, "{}_{}".format("trace_byte", "LNBGvsZJCZ_615.ts"))
video_format = 'mp4v'
fourcc = cv2.VideoWriter_fourcc(*video_format)
writer = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

frame_2_person_list = read_track_file(track_file)
for frame_idx in frame_2_person_list:
    if int(frame_idx) % 100 == 0:
        print(frame_idx)
    frame = cv2.imread('./datas/LNBGvsZJCZ_615.ts_imgs/{}.jpg'.format(frame_idx))
    person_list = frame_2_person_list[frame_idx]
    font = cv2.FONT_HERSHEY_SIMPLEX
    for person in person_list:
        person_id = person[0]
        person_x1, person_y1, person_w, person_h = person[1:5]
        color = get_color(person_id)
        cv2.rectangle(frame, (int(person_x1),int(person_y1)), (int(person_x1+person_w),int(person_y1+person_h)), color , 2)
        cv2.putText(frame, str(person_id), (int(person_x1-10),int(person_y1-10)), font, 1, (255,0,0), 3)
    writer.write(frame)
    
capture.release()
writer.release()